In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn

In [2]:
os.getcwd()

'/Users/oraff/Downloads'

In [3]:
df = pd.read_csv('AfterEDAChiCrime.csv')

In [4]:
df['Date'] = pd.to_datetime(df['Date'])

df.head()

,Unnamed: 0,ID,Date,Primary Type,Arrest,Domestic,District Name,Average Daily Wind Speed Mph,Fastest mile Wind Speed time,Peak gust time,...,Fog/Ice Fog,Heavy Fog,Thunder,Hail,Smoke/Haze,Tornado,Year,Month,Day_name,Severity
0,0,11109587,2001-01-01,DECEPTIVE PRACTICE,0,0,Rogers Park,7.38,1519.0,1518.0,...,0,0,0,0,0,0,2001,January,Monday,0
1,1,1780042,2001-01-01,OTHER OFFENSE,0,0,Morgan Park,7.38,1519.0,1518.0,...,0,0,0,0,0,0,2001,January,Monday,0
2,2,5691515,2001-01-01,THEFT,0,0,Englewood,7.38,1519.0,1518.0,...,0,0,0,0,0,0,2001,January,Monday,0
3,3,5233406,2001-01-01,THEFT,1,0,Near West,7.38,1519.0,1518.0,...,0,0,0,0,0,0,2001,January,Monday,1
4,4,1310513,2001-01-01,THEFT,0,0,Ogden,7.38,1519.0,1518.0,...,0,0,0,0,0,0,2001,January,Monday,0


In [5]:
df = df.drop('Unnamed: 0', axis=1)

In [6]:
df1 = df['Primary Type'].str.get_dummies()

In [7]:
df = pd.concat([df, df1], axis=1)

df.head()

,ID,Date,Primary Type,Arrest,Domestic,District Name,Average Daily Wind Speed Mph,Fastest mile Wind Speed time,Peak gust time,Percipitation,...,OTHER OFFENSE,PROSTITUTION,PUBLIC INDECENCY,PUBLIC PEACE VIOLATION,RITUALISM,ROBBERY,SEX OFFENSE,STALKING,THEFT,WEAPONS VIOLATION
0,11109587,2001-01-01,DECEPTIVE PRACTICE,0,0,Rogers Park,7.38,1519.0,1518.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,1780042,2001-01-01,OTHER OFFENSE,0,0,Morgan Park,7.38,1519.0,1518.0,0.0,...,1,0,0,0,0,0,0,0,0,0
2,5691515,2001-01-01,THEFT,0,0,Englewood,7.38,1519.0,1518.0,0.0,...,0,0,0,0,0,0,0,0,1,0
3,5233406,2001-01-01,THEFT,1,0,Near West,7.38,1519.0,1518.0,0.0,...,0,0,0,0,0,0,0,0,1,0
4,1310513,2001-01-01,THEFT,0,0,Ogden,7.38,1519.0,1518.0,0.0,...,0,0,0,0,0,0,0,0,1,0


In [8]:
df2 = df['District Name'].str.get_dummies()

In [9]:
df = pd.concat([df, df2], axis=1)

df.head()

,ID,Date,Primary Type,Arrest,Domestic,District Name,Average Daily Wind Speed Mph,Fastest mile Wind Speed time,Peak gust time,Percipitation,...,Lincoln,Morgan Park,Near North,Near West,Ogden,Rogers Park,Shakespeare,South Chicago,Town Hall,Wentworth
0,11109587,2001-01-01,DECEPTIVE PRACTICE,0,0,Rogers Park,7.38,1519.0,1518.0,0.0,...,0,0,0,0,0,1,0,0,0,0
1,1780042,2001-01-01,OTHER OFFENSE,0,0,Morgan Park,7.38,1519.0,1518.0,0.0,...,0,1,0,0,0,0,0,0,0,0
2,5691515,2001-01-01,THEFT,0,0,Englewood,7.38,1519.0,1518.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,5233406,2001-01-01,THEFT,1,0,Near West,7.38,1519.0,1518.0,0.0,...,0,0,0,1,0,0,0,0,0,0
4,1310513,2001-01-01,THEFT,0,0,Ogden,7.38,1519.0,1518.0,0.0,...,0,0,0,0,1,0,0,0,0,0


In [10]:
df3 = df['Day_name'].str.get_dummies()

df = pd.concat([df, df3], axis=1)

df.head()

,ID,Date,Primary Type,Arrest,Domestic,District Name,Average Daily Wind Speed Mph,Fastest mile Wind Speed time,Peak gust time,Percipitation,...,South Chicago,Town Hall,Wentworth,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,11109587,2001-01-01,DECEPTIVE PRACTICE,0,0,Rogers Park,7.38,1519.0,1518.0,0.0,...,0,0,0,0,1,0,0,0,0,0
1,1780042,2001-01-01,OTHER OFFENSE,0,0,Morgan Park,7.38,1519.0,1518.0,0.0,...,0,0,0,0,1,0,0,0,0,0
2,5691515,2001-01-01,THEFT,0,0,Englewood,7.38,1519.0,1518.0,0.0,...,0,0,0,0,1,0,0,0,0,0
3,5233406,2001-01-01,THEFT,1,0,Near West,7.38,1519.0,1518.0,0.0,...,0,0,0,0,1,0,0,0,0,0
4,1310513,2001-01-01,THEFT,0,0,Ogden,7.38,1519.0,1518.0,0.0,...,0,0,0,0,1,0,0,0,0,0


In [11]:
df4 = df['Month'].str.get_dummies()

df = pd.concat([df, df4], axis=1)

df.head()

,ID,Date,Primary Type,Arrest,Domestic,District Name,Average Daily Wind Speed Mph,Fastest mile Wind Speed time,Peak gust time,Percipitation,...,December,February,January,July,June,March,May,November,October,September
0,11109587,2001-01-01,DECEPTIVE PRACTICE,0,0,Rogers Park,7.38,1519.0,1518.0,0.0,...,0,0,1,0,0,0,0,0,0,0
1,1780042,2001-01-01,OTHER OFFENSE,0,0,Morgan Park,7.38,1519.0,1518.0,0.0,...,0,0,1,0,0,0,0,0,0,0
2,5691515,2001-01-01,THEFT,0,0,Englewood,7.38,1519.0,1518.0,0.0,...,0,0,1,0,0,0,0,0,0,0
3,5233406,2001-01-01,THEFT,1,0,Near West,7.38,1519.0,1518.0,0.0,...,0,0,1,0,0,0,0,0,0,0
4,1310513,2001-01-01,THEFT,0,0,Ogden,7.38,1519.0,1518.0,0.0,...,0,0,1,0,0,0,0,0,0,0


In [12]:
df['Hour'] = df.Date.dt.hour
df['Day_of_year'] = df.Date.dt.dayofyear
df['Quarter'] = df.Date.dt.quarter

from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

cal = calendar()
holidays = cal.holidays(start=df['Date'].min(), end=df['Date'].max())

df['Holiday'] = df['Date'].isin(holidays)


In [13]:
df = df.drop(columns=['Primary Type', 'District Name', 'Month', 'Day_name', 'Date'], axis=1)

In [14]:
df.head()

,ID,Arrest,Domestic,Average Daily Wind Speed Mph,Fastest mile Wind Speed time,Peak gust time,Percipitation,Snowfall,Snow depth,Average Temp,...,June,March,May,November,October,September,Hour,Day_of_year,Quarter,Holiday
0,11109587,0,0,7.38,1519.0,1518.0,0.0,0.0,17.0,20.0,...,0,0,0,0,0,0,0,1,1,True
1,1780042,0,0,7.38,1519.0,1518.0,0.0,0.0,17.0,20.0,...,0,0,0,0,0,0,0,1,1,True
2,5691515,0,0,7.38,1519.0,1518.0,0.0,0.0,17.0,20.0,...,0,0,0,0,0,0,0,1,1,True
3,5233406,1,0,7.38,1519.0,1518.0,0.0,0.0,17.0,20.0,...,0,0,0,0,0,0,0,1,1,True
4,1310513,0,0,7.38,1519.0,1518.0,0.0,0.0,17.0,20.0,...,0,0,0,0,0,0,0,1,1,True


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6519152 entries, 0 to 6519151
Columns: 102 entries, ID to Holiday
dtypes: bool(1), float64(11), int64(90)
memory usage: 4.9 GB


In [17]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

In [18]:
df_severe = df[df.Severity != 0]
df_non = df[df.Severity != 1]
df_severe_70 = df_severe.sample(n=70000)
df_non_70 = df_non.sample(n=70000)
df_tnt = df_severe_70.append(df_non_70)

In [19]:
X = df_tnt.drop(columns='Severity', axis=1)
y = df_tnt['Severity']


In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=1)


In [21]:
xg_cl = xgb.XGBClassifier(objective='binary:logistic', n_estimators=10, seed=1)

In [22]:
xg_cl.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=10, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=1,
              silent=None, subsample=1, verbosity=1)

In [23]:
preds = xg_cl.predict(X_test)

In [24]:
accuracy = float(np.sum(preds==y_test))/y_test.shape[0]
print("accuracy: %f" % (accuracy))


accuracy: 0.986929
